Próba implementacji metod zastosowanych przez dr. Kontka w artykule 

["**Weryfikacja wyniku drugiej tury wyborów prezydenckich w Polsce w 2025 roku: Przeliczenie głosów z użyciem przestrzennie grupowanej metody MAD**"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441)

dla wyników wyborów prezydenckich z 2015 roku

In [26]:
import pandas as pd

from utilities import presidential_data, clustering, methods

### load presidential both rouds - official PKW data

In [27]:
year = "2015"
ext = "xls"
df_2015_r1 = presidential_data.get_df(year, "1", ext)


df_2015_r2 = presidential_data.load_data(year, "2", ext)
df_2015_r2 = df_2015_r2.rename(columns={
    "Teryt Gminy": "teryt_gmina",
    "Numer obwodu": "polling_station_id"
})

# In 2015 round 2 data there is no postal code, so we need to join it to round 1
df_2015_r2 = df_2015_r2.merge(
    df_2015_r1[["teryt_gmina", "polling_station_id", "postal_code"]],
    on=["teryt_gmina", "polling_station_id"],
    how="left"
)
df_2015_r2 = presidential_data.process_df(df_2015_r2, year)

### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [28]:
cand_A = "komorowski"
cand_B = "duda"

df_2015 = presidential_data.join_both_rounds(cand_A, cand_B, df_2015_r1, df_2015_r2)

In [29]:
# grupowanie geograficzne metodą p. Białka
df_2015 = clustering.add_bialek_postal_buckets(df_2015)

### 1. pop_outlier: Nadmierne poparcie dla kandydata A (względem mediany w ramach lokalnej grupy)

In [30]:
print("Liczba komisji: ", len(df_2015)) # bez zagranicy i statków
df = df_2015.copy()

Liczba komisji:  26805


In [31]:
df = methods.add_anomaly_1(df, cand_A, cand_B, new_col_name="k_score_1")

In [32]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
komorowski: 3206
duda: 2928
---
k > 2.5
komorowski: 2298
duda: 2038
---
k > 3.0
komorowski: 1718
duda: 1464
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Duda jest 2928 a Komorowski 3206

2025: 3750 i 4551

2020: 2893 i 3430

### 2. growth_outlier: Nadmierny względny wzrost poparcia dla kandydata A między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla kandydata B w tej samej grupie lokalnej;

In [33]:
df = methods.add_anomaly_2(df, cand_A, cand_B, new_col_name="k_score_2")

In [34]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
komorowski: 2754
duda: 3074
---
k > 2.5
komorowski: 1950
duda: 2262
---
k > 3.0
komorowski: 1464
duda: 1707
---


k > 2.0

2025: trzaskowski: 3552 nawrocki: 3127

2020: trzaskowski: 3795 duda: 1863

### 3. Komisje, w których kand A uzyskał więcej głosów niż kand B w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę kand B;

In [35]:
df = methods.add_anomaly_3(df, cand_A, cand_B, new_col_name="flip")

In [36]:
# na korzyść kandydata A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('komorowski', 1806)

vs 2025:

('trzaskowski', 2608)

2020:

('trzaskowski', 1885)

In [37]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('duda', 1750)

vs 2025:

('nawrocki', 1843)

2020:

('duda', 1999)

In [38]:
df = df.drop(
    columns=[
        f"higher_median_{cand_A}",
        f"higher_median_{cand_B}",
    ]
)

**WYNIKI**:


W grupach, w których większą medianę miał Duda, było 1750 komisji, w których wyższy wynik uzyskał Komorowski.

W grupach, w których większą medianę miał Komorowski, było 1806 komisji, w których wyższy wyniki uzyskał Duda.

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [39]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('komorowski', 286)

W 106 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

vs 2025:

('trzaskowski', 128)

2020:

('trzaskowski', 106)

W 286 komisjach Komorowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

In [40]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('duda', 93)

W 284 (2025 - 112) komisjach Duda uzyskał mniej głosów w drugiej turze niż w pierwszej.


vs 2025:

('nawrocki', 112)

vs 2020:

('duda', 284)

W 93 komisjach Duda uzyskał mniej głosów w drugiej turze niż w pierwszej.

Przykładowe anomalie na korzyść Komorowskiego:

In [41]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,komorowski_r1,duda_r1,postal_code,komorowski_r2,duda_r2,postal_clean,bucket,komorowski_k_score_1,duda_k_score_1,komorowski_k_score_2,duda_k_score_2,komorowski_flip,duda_flip
1486,22605,1,71,114,59-516,196,112,59516,2316,0.666667,-1.130233,7.127835,-7.127835,True,False
12758,143406,4,88,104,05-280,169,101,05280,76,1.126316,-1.836538,7.651702,-7.651702,True,False
1273,22308,2,0,94,55-010,0,93,55010,1310,-3.602410,-2.813559,NaN,NaN,False,True
4773,61408,15,58,52,24-150,57,46,24150,484,-0.843750,-1.572770,0.775610,-0.775610,True,False
26199,302903,22,7,51,64-200,7,50,64200,1515,-2.517787,-2.015748,-0.040018,0.040018,False,True


In [42]:
# Anomalie na korzysc
df[cand_A + '_more_votes'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_more_votes'] = df[cand_A + '_r2']<df[cand_A + '_r1']

## Wyniki

In [43]:
# w zależności od wielkości k
k = 2

# deduplikacja, czyli jak wystąpiła więcej niz jedna anomalia, to liczymy taką komisję raz

### Na korzyść Komorowskiego

In [44]:
cand = "komorowski"
opponent = "duda"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [45]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       3206
growth_outlier    2754
flip              1806
more_votes          93
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (7073) albo wszystkie cztery anomalie (0)

In [46]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 7073
>=2: 756
>=3: 30
>=4: 0


### Próba odwzorowania tabelki z wynikami

In [47]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [48]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [49]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [50]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy komorowski,głosy duda,różnica przed,głosy komorowski po,głosy duda po,różnica po,zmiana
0,pop_outlier,2673,1177550,1240127,-62577,1065810,1351867,-286057,-223480
1,growth_outlier,2521,552905,681380,-128475,574879,659406,-84527,43948
2,flip,1786,570633,474382,96251,474499,570516,-96017,-192268
3,more_votes,93,2339,1635,704,1850,2124,-274,-978
4,łącznie,7073,2303427,2397524,-94097,2117038,2583913,-466875,-372778


### Na korzyść Dudy

In [51]:
cand = "duda"
opponent = "komorowski"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [52]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       2928
growth_outlier    3074
flip              1750
more_votes         286
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (7124) albo wszystkie cztery anomalie (1)

In [53]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 7124
>=2: 842
>=3: 71
>=4: 1


In [54]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [55]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [56]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [57]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy duda,głosy komorowski,różnica przed,głosy duda po,głosy komorowski po,różnica po,zmiana
0,pop_outlier,2569,1306296,1035245,271051,1292828,1048713,244115,-26936
1,growth_outlier,2584,599896,643513,-43617,624838,618571,6267,49884
2,flip,1685,454708,377982,76726,382703,449987,-67284,-144010
3,more_votes,286,19249,5122,14127,17380,6991,10389,-3738
4,łącznie,7124,2380149,2061862,318287,2317749,2124262,193487,-124800
